## 104人力銀行網站爬蟲

In [1]:
import requests
from bs4 import BeautifulSoup
import time

# 設定模擬的 headers
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
    'Referer': 'https://www.104.com.tw/company/uboh3w8?job=&roleJobCat=0_0&area=0&page=1&pageSize=20&order=8&asc=0&jobsource=checkc&tab=job',
    'Connection': 'keep-alive'
}

# 要爬取的職位列表頁面 URL
url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=1012001000&isJobList=1&jobcat=2007000000&jobsource=m_joblist_search&page=1&pagesize=20&searchJobs=1' #更改pagesize 可以獲得更多職業



# 發送請求
response = requests.get(url, headers=headers)

time.sleep(3)

# 檢查請求是否成功
if response.status_code == 200:
    # 使用 BeautifulSoup 解析頁面
    data = response.json()
    counter =0
    for job in data["data"]:
        print(job["jobName"])
        print(job["custName"])
        print (counter)
        counter+=1
    print(data)
    

else:
    print(f"Failed to retrieve data from {url}, status code: {response.status_code}")


網路管理工程師
聚贏有限公司
0
後端工程師
聚贏有限公司
1
客戶關係管理經理 CSM Manager
裕利股份有限公司
2
前端工程師 (新北市)
德煌生醫科技股份有限公司
3
【IT】RD_軟體前端工程師 (Frontend Developer)
科林儀器股份有限公司
4
【台北總部】雲端數據主管(數位資訊部)
淨妍醫美集團_淨妍生物科技股份有限公司
5
【RD】軟體前端工程師 (Frontend Developer)
濰視眼科診所
6
後端工程師 (新北市)
德煌生醫科技股份有限公司
7
資訊經理
馬光中醫診所
8
【正職】健管部-數據分析專員
博聯生技股份有限公司
9
資訊工程師
佑民醫療社團法人佑民醫院
10
【國泰集團】系統分析師
國泰健康管理顧問股份有限公司
11
資訊工程師
臺北榮民總醫院
12
醫學資訊室臨床資訊組程式設計師#6256
三軍總醫院附設民眾診療服務處_人事室
13
【IT】資料工程師(Data Engineer)
科林儀器股份有限公司
14
全端工程師 (新北市)
德煌生醫科技股份有限公司
15
總院 資訊部 助理程式師(約聘)
國泰醫療財團法人國泰綜合醫院
16
世博診所總務資訊
泰安診所
17
資訊部後端工程師
醫療財團法人辜公亮基金會和信治癌中心醫院
18
【杏一醫療用品★福利優】營運系統管理師_AKA
杏一醫療用品股份有限公司
19
【IT】RD_軟體後端工程師 (Backend Developer)
科林儀器股份有限公司
20
數據整合與顧客關係專員
良視眼科診所
21
{'data': [{'appearDate': '20250114', 'applyCnt': 31, 'coIndustry': 1001001003, 'coIndustryDesc': '網際網路相關業', 'custName': '聚贏有限公司', 'custNo': '130000000158555', 'description': '我們在尋找【具備團隊合作精神、勇於接受挑戰、熱愛學習】的網路管理工程師，負責規劃和組職主機網路架構，確保網路順暢和安全並與團隊合作完成產品開發。這裡有高度自由的辦公環境，期待值得信賴、積極學習的你加入，與我們一起解決複雜且有趣的技術挑戰，推動創新的產品開發！\n\n<工作內容>\n\n1. 規劃和組織主機網路架構

#### 抓取各式職缺

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [3]:


def get_all_job_info(indcat="1012001000", jobcat="2007000000", pagesize=30):

    # 設定模擬的 headers
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'Referer': 'https://www.104.com.tw',
        'Connection': 'keep-alive'
    }

    # API URL
    api_url = 'https://www.104.com.tw/jobs/search/api/jobs?indcat=' + indcat + '&isJobList=3&jobcat=' + jobcat + '&jobsource=m_joblist_search&page=1&pagesize=' + str(pagesize) + '&searchJobs=1'

    # 發送 API 請求
    response = requests.get(api_url, headers=headers)


    if response.status_code == 200:
        data = response.json()
        print(data)
        
        job_list = []
        counter = 0
        for job in data["data"]:
            job_name = job["jobName"]
            cust_name = job["custName"]
            job_link = job["link"]["job"]

            print(f"{counter}. Job Name: {job_name}, Company: {cust_name}, URL: {job_link}")
            job_list.append({"Job Name": job_name, "Company": cust_name, "URL": job_link})
            
            
            # 嘗試抓取薪資資訊
            try:
                job_response = requests.get(job_link, headers=headers)
                if job_response.status_code == 200:
                    job_soup = BeautifulSoup(job_response.text, 'lxml')

                    # 使用正則表達式搜尋薪資資訊
                    salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元')) 
                    if salary_tag:
                        salary_info = salary_tag.strip()  # 去除多餘空白
                        print(f"   Salary: {salary_info}")
                    else:
                        # 若找不到明確的薪資範圍，則顯示其他資訊
                        salary_tag = job_soup.find('div', class_='job-description__salary')
                        if salary_tag:
                            salary_info = salary_tag.get_text(strip=True)
                            print(f"   Salary: {salary_info}")
                        else:
                            print("   Salary: Not found")
            except Exception as e:  # 加入 except 區塊處理異常
                print(f"   Error fetching salary info: {e}")

            counter += 1
            # 避免被封鎖，延遲請求
            time.sleep(1)
        jobs_df = pd.DataFrame(job_list)
        return jobs_df

    else:
        print(f"Failed to retrieve data from {api_url}, status code: {response.status_code}")


#### 抓取工作的詳細資料

In [4]:
import requests
from lxml import html
import json

# 定義 URL 和 Headers
def get_job_detail(job_code):
    url = "https://www.104.com.tw/job/ajax/content/" + job_code
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36",
        "Referer": "https://www.104.com.tw/job/" + job_code
    }
    print(url)
    try:
        # 發送 GET 請求
        response = requests.get(url, headers=headers)
        
        # 檢查狀態碼
        if response.status_code == 200:
            # 將返回的 JSON 資料解析為 Python 字典
            data = response.json()
            print(data)
            # 提取需要的字段
            job_name = data.get('data', {}).get('header', {}).get('jobName', 'Not found')
            company_name = data.get('data', {}).get('header', {}).get('custName', 'Not found')
            salary = data.get('data', {}).get('jobDetail', {}).get('salary', 'Not found')
            job_description = data.get('data', {}).get('jobDetail', {}).get('jobDescription', 'Not found')
            job_welfare = data.get('data', {}).get('welfare', {}).get('welfare', 'Not found')
            # legal_tags = data.get('data', {}).get('legalTag', [])
            # other_welfare_tags = data.get('data', {}).get('welfare', {}).get('tag', [])
            # 整理輸出格式
            # print("Job Information")
            # print("====================")
            # print(f"Job Name       : {job_name}")
            # print(f"Company Name   : {company_name}")
            # print(f"Salary         : {salary}")
            # print(f"Description    :\n{job_description}")
            # print(f"Welfare    :\n{job_welfare}")
            return {
                "Job Name": job_name,
                "Company Name": company_name,
                "Salary": salary,
                "Description": job_description,
                "Welfare": job_welfare
                # "Legal_tags": legal_tags,
                # "other_Welfare_tags": other_welfare_tags
            }
            
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")


get_job_detail("6cbqb")

https://www.104.com.tw/job/ajax/content/6cbqb
{'data': {'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '總院 資訊部  程式師(網路工程師)', 'appearDate': '2025/01/14', 'custName': '國泰醫療財團法人國泰綜合醫院', 'custUrl': 'https://www.104.com.tw/company/1waq7ko', 'analysisType': 1, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/6cbqb', 'isSaved': False, 'isApplied': False, 'applyDate': '', 'userApplyCount': 0, 'isActivelyHiring': False}, 'contact': {'hrName': '彭組長', 'email': 'jason_peng@cgh.org.tw', 'visit': '', 'phone': ['02-27082121-1596'], 'other': '本院依菸害防制法，推動無菸職場，院區及院區外10公尺內全面實施禁菸', 'reply': ''}, 'environmentPic': {'environmentPic': [], 'corpImageBottom': {'imageUrl': 'https://static.104.com.tw/b_profile/cust_picture/9000/4129719000/custintroduce/image1.jpg?v=20240418150259', 'link': 'https://www.cgh.org.tw/'}}, 'condition': {'acceptRole': {'role': [], 'disRole': {'needHandicapCompendium': False, 'disability': []}}

{'Job Name': '總院 資訊部  程式師(網路工程師)',
 'Company Name': '國泰醫療財團法人國泰綜合醫院',
 'Salary': '月薪45,600元以上',
 'Description': '1. 網路資安設備（Firewall、WAF、Router、Switch）管理/維護/障礙排除\n2. 無線網路設備（Ruckus等）管理/維護/障礙排除\n3. 網路可靠度監測系統管理/維護\n4. 其他與網路架設相關之工作項目\n5. 支援分院建置維運網路設備\n6. 協助電腦設備故障排除\n\n\n\n                 ',
 'Welfare': '本院正職員工享有健全福利制度\n1.提供完善的在職教育訓練及國內外各項研習、進修課程機會\n2.享有勞保、健保、團保(含眷屬) \n3.年終獎金、端午代金、中秋代金、婚喪禮金\n4.子女教育補助費、國內外旅遊補助及康樂社團活動等\n5.年終餐會補助、生日贈點商品\n6.關係企業商品優惠、企業購屋優惠、企業員工房貸優惠及企業壽險保費優惠\n7.員工及眷屬、退休員工就醫優待\n8.午餐或晚餐便當費用補助\n9.定點交通車接駁服務\n10.員工托兒及托嬰服務'}

#### 工程師

In [5]:
engineer_jobs_df =  get_all_job_info(indcat="1012001001", jobcat="2007000000", pagesize=50)
engineer_jobs_df

{'data': [{'appearDate': '20250114', 'applyCnt': 31, 'coIndustry': 1001001003, 'coIndustryDesc': '網際網路相關業', 'custName': '聚贏有限公司', 'custNo': '130000000158555', 'description': '我們在尋找【具備團隊合作精神、勇於接受挑戰、熱愛學習】的網路管理工程師，負責規劃和組職主機網路架構，確保網路順暢和安全並與團隊合作完成產品開發。這裡有高度自由的辦公環境，期待值得信賴、積極學習的你加入，與我們一起解決複雜且有趣的技術挑戰，推動創新的產品開發！\n\n<工作內容>\n\n1. 規劃和組織主機網路架構，確保網路順暢和安全。\n2. 實施網路安全防護措施，防範未經授權的訪問和潛在威脅。\n3. 管理和監控數千台位於不同 IDC 的主機（VPS、VDS），確保其穩定運行。\n4. 執行系統建置、更新，以及生產環境的部署和維護。\n5. 設置和維護主機監控系統，及時發現並解決潛在問題。\n6. 管理公司內部主機，確保內部系統的正常運作。\n\n<必要條件>\n\n1. 具備對網路的基本觀念與認識（Gateway、DNS、Domain、IP、Proxy、VPN ）。\n2. 具備網路安全、資訊設備環境設定、伺服器網站管理維護等技能。\n3. 具備架設Nginx伺服器實務經驗（如 安全性設定、代理設定、負載平衡、緩存）。\n4. 熟悉 Linux、Shell 、Firewalll、Vmware 等工具的操作與維護。\n5. 有容器技術（如 Docker, Kubernetes）的實務工作經歷。\n6. 具備良好的問題解決能力和團隊合作精神。\n\n<加分條件>\n\n1. 具數位行銷經驗者優先，如 SEO、流量創建、轉換優化、A/B 測試等。\n2. 有 Python 相關實作經驗。\n3. 熟悉 CI/CD流程和工具，具備自動化實務經驗。\n4. 對人工智慧技術充滿熱情，積極學習並緊跟行業趨勢。', 'descSnippet': '我們在尋找【具備團隊合作精神、勇於接受挑戰、熱愛學習】的網路管理工程師，負責規劃和組職主機網路架構，確保網路順暢和安全並與團隊合作完成產品開發。這裡有高度自由的辦公環境，期待值得信賴、積極學

C:\Users\cah\AppData\Local\Temp\ipykernel_5724\4167284370.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元'))


1. Job Name: 後端工程師, Company: 聚贏有限公司, URL: https://www.104.com.tw/job/8l4mm
   Salary: Not found
2. Job Name: 資訊工程師, Company: 佑民醫療社團法人佑民醫院, URL: https://www.104.com.tw/job/72lo7
   Salary: Not found
3. Job Name: 資訊工程師, Company: 臺北榮民總醫院, URL: https://www.104.com.tw/job/8kpkd
   Salary: Not found
4. Job Name: 醫學資訊室臨床資訊組程式設計師#6256, Company: 三軍總醫院附設民眾診療服務處_人事室, URL: https://www.104.com.tw/job/6wbrk
   Salary: Not found
5. Job Name: 總院 資訊部 助理程式師(約聘), Company: 國泰醫療財團法人國泰綜合醫院, URL: https://www.104.com.tw/job/7qhzv
   Salary: Not found
6. Job Name: 資訊部後端工程師, Company: 醫療財團法人辜公亮基金會和信治癌中心醫院, URL: https://www.104.com.tw/job/830wa
   Salary: Not found
7. Job Name: 資訊部前端工程師, Company: 醫療財團法人辜公亮基金會和信治癌中心醫院, URL: https://www.104.com.tw/job/830wj
   Salary: Not found
8. Job Name: 總院 資訊部  程式師(網路工程師), Company: 國泰醫療財團法人國泰綜合醫院, URL: https://www.104.com.tw/job/6cbqb
   Salary: Not found
9. Job Name: 總院 資訊部  約聘助理程式師(系統維護/操作人員), Company: 國泰醫療財團法人國泰綜合醫院, URL: https://www.104.com.tw/job/7ku7p
   Salary: Not found

,Job Name,Company,URL
0,網路管理工程師,聚贏有限公司,https://www.104.com.tw/job/8flaz
1,後端工程師,聚贏有限公司,https://www.104.com.tw/job/8l4mm
2,資訊工程師,佑民醫療社團法人佑民醫院,https://www.104.com.tw/job/72lo7
3,資訊工程師,臺北榮民總醫院,https://www.104.com.tw/job/8kpkd
4,醫學資訊室臨床資訊組程式設計師#6256,三軍總醫院附設民眾診療服務處_人事室,https://www.104.com.tw/job/6wbrk
5,總院 資訊部 助理程式師(約聘),國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/job/7qhzv
6,資訊部後端工程師,醫療財團法人辜公亮基金會和信治癌中心醫院,https://www.104.com.tw/job/830wa
7,資訊部前端工程師,醫療財團法人辜公亮基金會和信治癌中心醫院,https://www.104.com.tw/job/830wj
8,總院 資訊部 程式師(網路工程師),國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/job/6cbqb
9,總院 資訊部 約聘助理程式師(系統維護/操作人員),國泰醫療財團法人國泰綜合醫院,https://www.104.com.tw/job/7ku7p


In [6]:
import time
job_details_list = []
for url in engineer_jobs_df['URL']:
    job_code = url.split('/')[-1]
    print(job_code)
    job_details = get_job_detail(job_code)
    job_details_list.append(job_details)
    time.sleep(2)
    
job_details_df = pd.DataFrame(job_details_list)
job_details_df.to_csv('engineer_job_details.csv', index=False, encoding='utf-8-sig')

8flaz
https://www.104.com.tw/job/ajax/content/8flaz
{'data': {'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '網路管理工程師', 'appearDate': '2025/01/14', 'custName': '聚贏有限公司', 'custUrl': 'https://www.104.com.tw/company/1a2x6bldnf', 'analysisType': 4, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/8flaz', 'isSaved': False, 'isApplied': False, 'applyDate': '', 'userApplyCount': 0, 'isActivelyHiring': True}, 'contact': {'hrName': 'Wish', 'email': '', 'visit': '', 'phone': [], 'other': '', 'reply': '合適者將於5個工作天內主動聯繫，不合適者將不另行通知'}, 'environmentPic': {'environmentPic': [{'thumbnailLink': 'https://static.104.com.tw/b_profile/cust_picture/8555/130000000158555/env/s_892440735961026152.jpg?v=20240809170911', 'link': 'https://static.104.com.tw/b_profile/cust_picture/8555/130000000158555/env/l_892440735961026152.jpg?v=20240809170911', 'description': ''}, {'thumbnailLink': 'https://static.104.com.tw/b_profile/cus

#### 護理師

In [7]:
nurse_jobs_df =  get_all_job_info( indcat="1012001001", jobcat="2015001004",pagesize=50)
nurse_jobs_df

{'data': [{'appearDate': '20250109', 'applyCnt': 3, 'coIndustry': 1012001001, 'coIndustryDesc': '醫院', 'custName': '宏恩醫療財團法人宏恩綜合醫院', 'custNo': '4173290000', 'description': '手術室護理人員(沒有大夜班)\n1.手術室護理工作\n2.主管交辦事項\n3.一般護理工作事項\n4.績效獎金另計\n5.加班費依勞基法規範另計\n6.薪資待遇從優\n\n病房護理人員(無經驗可)\n1. 提供基本的病患基礎醫藥照顧， 如：包紮傷口、換藥等\n2. 測量並且記錄病患的生理狀態（如：身高、體重、體溫、血壓、脈搏和呼吸）、進食量、排便狀況等\n3. 進行靜脈注射、肌肉注射、打點滴，觀察記錄患者給藥之後的狀況並加以回報\n4. 觀察、記錄並回報病人在接受醫療後狀態的改變，特別是一些不尋常的症狀反應，必要時採取救護措施\n5. 接聽病患求助電話，並且提供協助\n6. 協助病患保持個人衛生、更衣、移動、站立或行走\n7. 評估護理情形，在必要時與醫生會診\n8. 指導病患居家自我療護，如：解釋醫囑、術前術後衛教，並且提供相關諮詢\n9. 與病人及其家屬建立良好之人際關係，以減少不必要的誤會糾紛\n10. 督導其他護士及助理的護理工作\n11.績效獎金另計\n12.加班費依勞基法規範另計\n13.薪資待遇從優\n\n麻醉護理師\n1、通過麻醉相關訓練且領有證書，並可獨立完成工作\n2、執行臨床麻醉護理工作\n3、協助麻醉後照護及相關業務\n4、協助疼痛控制相關業務\n5、其他主管交辦業務\n6、加班費依勞基法另計\n7、薪資待遇從優\n\n', 'descSnippet': '手術室護理人員(沒有大夜班)\n1.手術室護理工作\n2.主管交辦事項\n3.一般護理工作事項\n4.績效獎金另計\n5.加班費依勞基法規範另計\n6.薪資待遇從優\n\n病房護理人員(無經驗可)\n1. 提供基本的病患基礎醫藥照顧， 如：包紮傷口、換藥等\n2. 測量並且記錄病患的生理狀態（如：身高、體重、體溫、血壓、脈搏和呼吸）、進食量、排便狀況等\n3. 進行靜脈注射、肌肉注射、打點滴，觀察記錄患者給藥之後的狀況並加以回報\n4. 觀察、記錄並回

C:\Users\cah\AppData\Local\Temp\ipykernel_5724\4167284370.py:41: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  salary_tag = job_soup.find(text=re.compile(r'月薪\s*\d+,\d+~\d+,\d+\s*元'))


1. Job Name: 奇美醫院【總院】健康服務護理師, Company: 奇美醫療財團法人奇美醫院, URL: https://www.104.com.tw/job/85uz3
   Salary: Not found
2. Job Name: 門診護理師(可年後上班), Company: 天祥醫院(籌備處), URL: https://www.104.com.tw/job/88lk0
   Salary: Not found
3. Job Name: 居家護理師, Company: 豐榮醫院, URL: https://www.104.com.tw/job/87joy
   Salary: Not found
4. Job Name: 時薪高彈性兼職可固定時段,護理部門診區兼職護理人員暨護士助理(保障工時工作單純易上手), Company: 中英醫療社團法人中英醫院, URL: https://www.104.com.tw/job/66its
   Salary: Not found
5. Job Name: 血液透析技術員(需具血液透析證照), Company: 天主教靈醫會醫療財團法人羅東聖母醫院, URL: https://www.104.com.tw/job/3hhuq
   Salary: Not found
6. Job Name: 產房助產師, Company: 林新醫療社團法人林新醫院, URL: https://www.104.com.tw/job/8li49
   Salary: Not found
7. Job Name: 約用護理師(護理之家), Company: 國立陽明交通大學附設醫院, URL: https://www.104.com.tw/job/8ans9
   Salary: Not found
8. Job Name: 行政-社區暨長照服務部-出院準備管理師, Company: 佛教慈濟醫療財團法人台北慈濟醫院, URL: https://www.104.com.tw/job/78l2i
   Salary: Not found
9. Job Name: 門診護理人員, Company: 大里仁愛醫院_仁愛醫療財團法人, URL: https://www.104.com.tw/job/4ud53
   Salary: No

,Job Name,Company,URL
0,護理人員(無經驗可),宏恩醫療財團法人宏恩綜合醫院,https://www.104.com.tw/job/7ro2f
1,奇美醫院【總院】健康服務護理師,奇美醫療財團法人奇美醫院,https://www.104.com.tw/job/85uz3
2,門診護理師(可年後上班),天祥醫院(籌備處),https://www.104.com.tw/job/88lk0
3,居家護理師,豐榮醫院,https://www.104.com.tw/job/87joy
4,"時薪高彈性兼職可固定時段,護理部門診區兼職護理人員暨護士助理(保障工時工作單純易上手)",中英醫療社團法人中英醫院,https://www.104.com.tw/job/66its
5,血液透析技術員(需具血液透析證照),天主教靈醫會醫療財團法人羅東聖母醫院,https://www.104.com.tw/job/3hhuq
6,產房助產師,林新醫療社團法人林新醫院,https://www.104.com.tw/job/8li49
7,約用護理師(護理之家),國立陽明交通大學附設醫院,https://www.104.com.tw/job/8ans9
8,行政-社區暨長照服務部-出院準備管理師,佛教慈濟醫療財團法人台北慈濟醫院,https://www.104.com.tw/job/78l2i
9,門診護理人員,大里仁愛醫院_仁愛醫療財團法人,https://www.104.com.tw/job/4ud53


In [9]:
import time
job_details_list = []
for url in nurse_jobs_df['URL']:
    job_code = url.split('/')[-1]
    print(job_code)
    job_details = get_job_detail(job_code)
    job_details_list.append(job_details)
    time.sleep(2)
    
job_details_df = pd.DataFrame(job_details_list)
job_details_df.to_csv('nurse_jobs_details.csv', index=False, encoding='utf-8-sig')

7ro2f
https://www.104.com.tw/job/ajax/content/7ro2f
{'data': {'corpImageRight': {'corpImageRight': {'imageUrl': '', 'link': ''}}, 'header': {'corpImageTop': {'imageUrl': '', 'link': ''}, 'jobName': '護理人員(無經驗可)', 'appearDate': '2025/01/09', 'custName': '宏恩醫療財團法人宏恩綜合醫院', 'custUrl': 'https://www.104.com.tw/company/1x0o368', 'analysisType': 1, 'analysisUrl': '//www.104.com.tw/jobs/apply/analysis/7ro2f', 'isSaved': False, 'isApplied': False, 'applyDate': '', 'userApplyCount': 0, 'isActivelyHiring': False}, 'contact': {'hrName': '林小姐', 'email': '', 'visit': '台北市大安區仁愛路四段61號', 'phone': ['2771-3161-706'], 'other': '', 'reply': '合適者將於7個工作天內主動聯繫，不合適者將不另行通知'}, 'environmentPic': {'environmentPic': [], 'corpImageBottom': {'imageUrl': '', 'link': ''}}, 'condition': {'acceptRole': {'role': [], 'disRole': {'needHandicapCompendium': False, 'disability': []}}, 'workExp': '不拘', 'edu': '高中以上', 'major': ['護理助產相關'], 'language': [], 'localLanguage': [], 'specialty': [], 'skill': [], 'certificate': [{'name': '